In [ ]:
import os
import requests
from dotenv import load_dotenv
import clickhouse_connect
import itertools
import pandas as pd
import json

load_dotenv()
clickhouse_host = os.getenv("CLICKHOUSE_HOST")
username = os.getenv("CLICKHOUSE_USER")
password = os.getenv("CLICKHOUSE_PASSWORD")
github_token = os.getenv("GITHUB_TOKEN")

client = clickhouse_connect.get_client(host=clickhouse_host, port=8123, username=username, password=password)

headers = {
  "Authorization": f"token {github_token}"
}

In [ ]:
# search most related projects
def search_projects(project_ids, limit):
    sql_search_projects = """
    WITH
        -- 获取仓库在指定时间段内的活跃开发者
        active_developers AS (
            SELECT DISTINCT actor_id, repo_id, repo_name
            FROM opensource.events
            WHERE repo_id IN (%s)
            AND (type IN ('IssuesEvent', 'PullRequestEvent', 'IssueCommentEvent', 'PullRequestReviewEvent','PullRequestReviewCommentEvent'))
        ),
        -- 计算其他仓库中的活跃开发者
        repo_activity AS (
            SELECT repo_id, repo_name, COUNT(DISTINCT actor_id) AS active_count
            FROM opensource.events
            WHERE actor_id IN (SELECT actor_id FROM active_developers)
            AND repo_id NOT IN (%s)
            AND (type IN ('IssuesEvent', 'PullRequestEvent', 'IssueCommentEvent','PullRequestReviewEvent', 'PullRequestReviewCommentEvent'))
            GROUP BY repo_id, repo_name
        )
        -- 获取关联度最高的 x 个仓库
        SELECT repo_id, repo_name, active_count
        FROM repo_activity
        ORDER BY active_count DESC
        LIMIT %s
        """
    formatted_query = sql_search_projects % (', '.join(f"'{id}'" for id in project_ids), ', '.join(f"'{id}'" for id in project_ids), limit)
    results = client.query(formatted_query)
    return results

In [ ]:
# fetch repo_name and avg OpenRank 25 from clickhouse
def execute_repo_details(repo_id):
  sql_query = """
    -- 根据 repo_id 查询最新的 repo_name 和 openrank 均值
    WITH latest_repo_name AS (
      SELECT repo_id, repo_name
      FROM opensource.events
      WHERE repo_id = %s
      ORDER BY created_at DESC
      LIMIT 1
    ),
    avg_openrank AS (
      SELECT repo_id, ROUND(avg(openrank), 0) AS average_openrank
      FROM opensource.global_openrank
      WHERE repo_id = %s
      AND platform = 'GitHub'
      AND created_at >= '2025-01-01'
      AND created_at < '2026-01-01'
      GROUP BY repo_id
    )
    SELECT n.repo_id, n.repo_name, o.average_openrank
    FROM latest_repo_name n
    JOIN avg_openrank o ON n.repo_id = o.repo_id
  """
  formatted_query = sql_query % (f"'{repo_id}'", f"'{repo_id}'")
  results = client.query(formatted_query)
  return results.result_rows

# fetch stars and descripiton through repo_name
def fetch_repo_info(repo_names, headers):
  github_repo_url = "https://api.github.com/repos/"
  repo_data = []

  for repo_name in repo_names:
    response = requests.get(github_repo_url + repo_name, headers=headers)
    if response.status_code == 200:
      data = response.json()
      repo_id = data['id']
      stars = data['stargazers_count']
      description = data['description']
      repo_data.append({'repo_name': repo_name, 'stars': stars, 'description': description, 'repo_id': repo_id})
    else:
      print(f"Failed to fetch data for {repo_name}")
  
  return repo_data

In [ ]:
# Read the initial repositories from the CSV file
initial_repos = pd.read_csv('initial_repos.csv')
iter1_repos = pd.read_csv('iter1_repos.csv')
iter2_repos = pd.read_csv('iter2_repos.csv')

# repo_id_full
repo_id_full = list(set(initial_repos['repo_id'].tolist() + iter1_repos['repo_id'].tolist() + iter2_repos['repo_id'].tolist()))

repo_ids = iter2_repos['repo_id'].tolist()

projects = set()
for repo_id in repo_ids:
    results = execute_search_projects([repo_id], 5)
    for row in results.result_rows:
        projects.add (row[0])
projects = list(projects)

unique_projects_ids = list(set(projects) - set(repo_id_full))